In [1]:
import time
import os
import math
import cv2
import numpy as np
import pandas as pd
import pyautogui as pa
import shutil
import imageio
from PIL import Image, ImageDraw
import random
from scipy.stats import norm, kstest, sem, ttest_ind, linregress
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from lib.analysis.cell_roi import RandomROISample
from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_DAT import AutoDAT
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
from lib.analysis.laminar_dist import *
from lib.file.TIF import *
from lib.analysis.align import ImageAlign
from lib.utilities import *
from lib.file.ROI_writer import ROIFileWriter
from lib.file.ROI_reader import ROIFileReader
from lib.analysis.barrel_roi import Barrel_ROI_Creator
from lib.analysis.false_discovery_control import false_discovery_control
from lib.analysis.trace_metrics import TraceMetrics

from lib.analysis.movie_maker import MovieMaker
###########################################
# Now we allow 3 barrels in a field of view to be analyzed
# in catalog, number of barrels is saved in Num_Barrels column
# The <slic>_<loc>_barrel_boundary.dat file is in ascending order of barrels, one
#   separate ROI for each barrel
###########################################

In [ ]:
date, slic, loc, rec, zda_subdir, subdir_2 =  "12-20-24", 1, 3, 1, "converted_zda", 'L23_vs_L4_NBQX/L4_NBQX_2'
# date, slic, loc, rec, zda_subdir, subdir_2  =  "10-29-24", 1, 1, 3, "selected_zda", 'L23_vs_L4_NBQX/L4_NBQX_2'
date, slic, loc, rec, zda_subdir, subdir_2 =  "10-23-24", 1, 2, 3, "selected_zda", 'L23_vs_L4_NBQX/L4_NBQX_2'
#date, slic, loc, rec, zda_subdir, subdir_2 =  "12-09-24", 1, 1, 1, "converted_zda", 'L23_vs_L4_NBQX/L4_NBQX'
data_dir = f"C:/Users/jjudge3/Desktop/Data/crossing_PPR/{subdir_2}/" + date + f"/{zda_subdir}/analysis0{slic}_0{loc}_0{rec}/"
trace_file = f"C:/Users/jjudge3/Desktop/Data/crossing_PPR/{subdir_2}/{date}/{zda_subdir}/_" + f"{slic}_{loc}_{rec}_trace_{slic}_rois.dat"
trace_supra_file = f"C:/Users/jjudge3/Desktop/Data/crossing_PPR/{subdir_2}/{date}/{zda_subdir}/" + f"supragranular_{slic}_{loc}_{rec}_trace_{slic}_rois_supragranular.dat"
ladder_df = 'C:/Users/jjudge3/Desktop/Data/crossing_PPR/export_summary_final_velocity_ladders_backup.csv'
ladder_traces = f"C:/Users/jjudge3/Desktop/Data/crossing_PPR/{subdir_2}/" + date + f"/{zda_subdir}/ladder_{slic}_{loc}_{rec}_trace_{slic}_rois_output__ladder_.dat"
ladder_df = pd.read_csv(ladder_df, sep=',', header=0)
start_frame = 94
end_frame = 200
frame_step_size = 1
print(data_dir)
def read_dat_trace_file(filename):
    data_arr = pd.read_csv(filename,
                                 sep='\t',
                                 header=0)
    return data_arr
# strip zeros from date day and month
formatted_date = pd.to_datetime(date).strftime('%m/%d/%Y').lstrip("0").replace("/0", "/")
print(formatted_date)
rec_ladder_df = ladder_df[(ladder_df['Date'] == formatted_date) & (ladder_df['Slice'] == slic) & (ladder_df['Location'] == loc) & (ladder_df['Recording'] == rec)]
rec_ladder_df

C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX_2/10-23-24/selected_zda/analysis01_02_03/
10/23/2024


,ROI,amp,snr,latency,halfwidth,Stim_Distance,X_Center,Y_Center,ROI_Set,Date,...,Coronal/Sagittal,IPI,Is_Pulse_2,Dist_from_home_barrel,resides_in_barrel,instantaneous_velocity,ROI file,Medial/Lateral,Age,Sex
504,1,0.133412,5.70521,4.1131,21.55310,236.753036,25.159574,9.329787,1_rois_output__ladder_,10/23/2024,...,NaN,NaN,False,499.134323,NaN,355.536009,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,Lateral,71.0,M
505,2,0.173699,6.76397,4.0658,10.52990,223.293529,26.053191,12.127660,1_rois_output__ladder_,10/23/2024,...,NaN,NaN,False,482.317469,NaN,290.417201,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,Lateral,71.0,M
506,3,0.218004,9.52456,4.0000,10.74210,204.792578,28.052083,14.614583,1_rois_output__ladder_,10/23/2024,...,NaN,NaN,False,463.208018,Neighbor,185.452227,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,Lateral,71.0,M
507,4,0.266509,12.79670,3.9050,11.32060,192.000000,29.413043,17.250000,1_rois_output__ladder_,10/23/2024,...,NaN,NaN,False,445.590056,Neighbor,329.282647,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,Lateral,71.0,M
508,5,0.305685,11.72310,3.8493,10.23820,174.000000,31.082474,19.814433,1_rois_output__ladder_,10/23/2024,...,NaN,NaN,False,427.249013,Neighbor,86.877526,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,Lateral,71.0,M
509,6,0.326727,15.83890,3.6565,9.88265,156.460858,31.967742,22.623656,1_rois_output__ladder_,10/23/2024,...,NaN,NaN,False,410.499026,Neighbor,128.890678,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,Lateral,71.0,M
510,7,0.333272,17.95690,3.5051,10.09170,145.986301,34.197917,25.020833,1_rois_output__ladder_,10/23/2024,...,NaN,NaN,False,390.984977,Neighbor,153.372688,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,Lateral,71.0,M
511,8,0.336888,18.17290,3.3979,10.89260,129.522199,35.000000,27.838710,1_rois_output__ladder_,10/23/2024,...,NaN,NaN,False,374.543425,Neighbor,934.362409,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,Lateral,71.0,M
512,9,0.350102,20.47490,3.3770,10.07830,113.841996,37.287234,30.191489,1_rois_output__ladder_,10/23/2024,...,NaN,NaN,False,355.015250,NaN,88.240638,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,Lateral,71.0,M
513,10,0.362973,20.36590,3.1952,14.54520,107.331263,37.914894,33.074468,1_rois_output__ladder_,10/23/2024,...,NaN,NaN,False,338.973102,NaN,153.999343,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,Lateral,71.0,M


In [3]:
trace_data = read_dat_trace_file(trace_file)
time = trace_data['Pt'].values * 0.5

trace_data_supra = read_dat_trace_file(trace_supra_file)

# vmax is max across all columns except 'Pt'
vmax = trace_data.drop(columns=['Pt']).max().max()
vmin = trace_data.drop(columns=['Pt']).min().min()

vmax_supra = trace_data_supra.drop(columns=['Pt']).max().max() - 1
vmin_supra = trace_data_supra.drop(columns=['Pt']).min().min() - 1

trace_data_ladder = read_dat_trace_file(ladder_traces)
vmin = min(vmin, vmin_supra)
stim_time = 96.6
trace_data_ladder


,Pt,ROI1,ROI2,ROI3,ROI4,ROI5,ROI6,ROI7,ROI8,ROI9,...,ROI19,ROI20,ROI21,ROI22,ROI23,ROI24,ROI25,ROI26,ROI27,ROI28
0,0,-0.049495,-0.047067,-0.025832,-0.000571,0.007798,-0.030309,-0.028524,-0.040770,-0.039764,...,0.002764,-0.000487,-0.003947,-0.025907,-0.051496,-0.087003,-0.081141,-0.077881,-0.070906,-0.082216
1,1,-0.057515,-0.055745,-0.036029,-0.019941,-0.014846,-0.045147,-0.035781,-0.050041,-0.045687,...,-0.023853,-0.018157,-0.024352,-0.043085,-0.066879,-0.090417,-0.085149,-0.094568,-0.080230,-0.085152
2,2,-0.064669,-0.061047,-0.041147,-0.034746,-0.031609,-0.054657,-0.036075,-0.053121,-0.046341,...,-0.045627,-0.034006,-0.045111,-0.054922,-0.077205,-0.087687,-0.082050,-0.108052,-0.082520,-0.081356
3,3,-0.071876,-0.063516,-0.040123,-0.043034,-0.039905,-0.057013,-0.030014,-0.049832,-0.043319,...,-0.058103,-0.045221,-0.059699,-0.059936,-0.081699,-0.080968,-0.073180,-0.115144,-0.077951,-0.071414
4,4,-0.078023,-0.063837,-0.033999,-0.045031,-0.040941,-0.053361,-0.022568,-0.044031,-0.040618,...,-0.059969,-0.049453,-0.061329,-0.058076,-0.080865,-0.074711,-0.063689,-0.113138,-0.069420,-0.059471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,894,-0.067309,-0.044799,-0.069945,-0.069169,-0.073213,-0.070580,-0.067889,-0.069060,-0.067333,...,-0.077778,-0.075610,-0.081086,-0.063581,-0.075672,-0.064521,-0.056697,-0.041395,-0.038636,-0.040612
895,895,-0.065001,-0.043823,-0.059335,-0.060188,-0.058418,-0.061599,-0.067037,-0.064878,-0.061435,...,-0.065930,-0.069972,-0.071434,-0.052753,-0.066402,-0.057492,-0.046866,-0.038018,-0.041287,-0.048927
896,896,-0.067090,-0.049224,-0.047069,-0.052253,-0.042750,-0.052334,-0.067155,-0.061519,-0.057799,...,-0.053543,-0.063293,-0.062159,-0.048913,-0.063328,-0.056804,-0.041272,-0.046346,-0.047274,-0.048765
897,897,-0.075192,-0.058736,-0.037211,-0.045281,-0.030245,-0.045506,-0.067314,-0.062109,-0.057367,...,-0.041454,-0.057896,-0.054840,-0.052030,-0.070102,-0.067410,-0.047223,-0.071138,-0.062543,-0.038671


In [4]:
# create a movie of a png image next to a trace plot
custom_frames = []
frame_ratio = 1
plot_type = 'trace'  # trace
min_lat = np.min(rec_ladder_df['latency'])
for frame in range(start_frame, end_frame, frame_step_size):
    for i_partial in range(frame_ratio):
        fig, ax = plt.subplots(1, 2, figsize=(10, 6))
        image_path = os.path.join(data_dir, f"{frame}.jpg")
        print(f"Processing image {frame}: {image_path}")
        
        # display the image with matplotlib
        last_img = img if 'img' in locals() else None
        try:
            img = plt.imread(image_path)
        except FileNotFoundError:
            img = last_img  # use the last valid image if current one is missing
        ax[0].imshow(img)
        ax[0].axis('off')  # Hide axes

        # right subplot for trace plot
        if plot_type == 'trace':
            for col in trace_data:
                if col != 'Pt':
                    ax[1].plot(time[:frame], -trace_data[col][:frame], label=col)
            for col in trace_data_supra:
                if col != 'Pt':
                    ax[1].plot(time[:frame], -trace_data_supra[col][:frame] + 1, label=col)
            ax[1].set_xlim(25, time[end_frame + 30])
            ax[1].set_ylim(-vmax, -vmin)
            ax[1].set_xlabel('Time (ms)')
            ax[1].set_ylabel('ΔF/F')
            ax[1].set_yticks([])
            
            # mark stim time with a vertical line
            ax[1].axvline(x=stim_time * 0.5, color='b', linestyle='--')
            x_ticks = [i + stim_time * 0.5 for i in [-20, 0, 20, 40]]
            ax[1].set_xlim(stim_time * 0.5 - 20, stim_time * 0.5 + 40)
            print(x_ticks)
            ax[1].set_xticks(x_ticks)
            ax[1].set_xticklabels([i for i in range(-20, 41, 20)])
            # remove spines
            ax[1].spines['top'].set_visible(False)
            ax[1].spines['right'].set_visible(False)

        elif plot_type == 'latency':
            # iterate through latencies and plot those whose time is less than the current frame
            num_pts_plotted = 0

            for index, row in rec_ladder_df.iterrows():
                lat = row['latency']
                
                if lat > 10:
                    continue
                elif lat * 2 + stim_time < float(frame) + float(i_partial) / float(frame_ratio):
                    dist = row['Dist_from_home_barrel']
                    # star marker
                    ax[1].scatter(dist, lat + 1, 
                                  color='tab:blue' if (lat < 1.8 or abs(dist-300) < 90) else 'tab:green' if lat < 2.1 else ('tab:orange' if dist > 300 else 'tab:green'),
                                  marker='*' if lat <= min_lat else 'o')
                    num_pts_plotted += 1
            print('num_pts_plotted:', num_pts_plotted)

            ax[1].set_xlim(50, 550)
            ax[1].set_ylim(-6, 0)
            ax[1].set_xlabel('Distance along L4 (μm)')
            ax[1].set_ylabel('Latency (ms)')
            ax[1].set_xticks([100, 200, 300, 400, 500])
            ax[1].set_xticklabels([-200, -100, 0, 100, 200])

            # mark the current time with a horizontal dotted line
            ax[1].axhline(y=(float(frame) + float(i_partial) / float(frame_ratio) - stim_time) * 0.5 + 1, color='black', linestyle='--')
            if i_partial == 0:
                plt.show()

        elif plot_type == 'trace-vert':
            # iterate through latencies and plot those whose time is less than the current frame
            num_pts_plotted = 0

            # plot all traces vertically at the position of their corresponding distance
            for col in trace_data_ladder:
                if col == 'Pt':
                    continue
                if col.startswith('ROI'):
                    roi_no = int(col[3:])
                
                # look up ROI num in rec_ladder_df
                dist = rec_ladder_df[rec_ladder_df['ROI'] == roi_no]['Dist_from_home_barrel'].values[0]
                lat = rec_ladder_df[rec_ladder_df['ROI'] == roi_no]['latency'].values[0]

                # take only points [stim_time, stim_time + 12] from trace_data
                trace_pts = trace_data_ladder[(trace_data_ladder['Pt'] < frame + i_partial / frame_ratio)]

                # plot the trace vertically at the position of their corresponding distance, connect points in scatter 
                ax[1].plot( trace_pts[col] * 10 + dist, (trace_pts['Pt'] - stim_time + 10) * 0.5, 
                           color='tab:blue' if (lat < 1.8 or abs(dist-300) < 90) else 'tab:green' if lat < 2.1 else ('tab:orange' if dist > 300 else 'tab:green'), 
                           alpha=0.5)


            ax[1].set_xlim(50, 550)
            ax[1].set_ylim(0, 20)
            ax[1].set_xlabel('Distance along L4 (μm)')
            ax[1].set_ylabel('Time (ms)')
            ax[1].set_xticks([100, 200, 300, 400, 500])
            ax[1].set_xticklabels([-200, -100, 0, 100, 200])

            # remove spines
            ax[1].spines['top'].set_visible(False)
            ax[1].spines['right'].set_visible(False)

        # save the figure to file
        output_path = os.path.join(data_dir, f"custom_frame_{frame}_{i_partial}.png")
        fig.savefig(output_path, bbox_inches='tight', pad_inches=0.1)
        plt.close(fig)  # Close the figure to free memory
        custom_frames.append(output_path)

# create a movie from the saved frames
custom_frames = [imageio.imread(frame) for frame in custom_frames]
imageio.mimsave(os.path.join(data_dir, 'custom_movie velocity v7.mp4'), custom_frames, fps=5)
print(f"Custom movie saved to {data_dir}custom_movie velocity v7.mp4")

Processing image 94: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX_2/10-23-24/selected_zda/analysis01_02_03/94.jpg
[28.299999999999997, 48.3, 68.3, 88.3]
Processing image 95: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX_2/10-23-24/selected_zda/analysis01_02_03/95.jpg
[28.299999999999997, 48.3, 68.3, 88.3]
Processing image 96: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX_2/10-23-24/selected_zda/analysis01_02_03/96.jpg
[28.299999999999997, 48.3, 68.3, 88.3]
Processing image 97: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX_2/10-23-24/selected_zda/analysis01_02_03/97.jpg
[28.299999999999997, 48.3, 68.3, 88.3]
Processing image 98: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX_2/10-23-24/selected_zda/analysis01_02_03/98.jpg
[28.299999999999997, 48.3, 68.3, 88.3]
Processing image 99: C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX_2/10-23-24/selected_zda/analysis01_02_03/99

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (803, 525) to (816, 528) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Custom movie saved to C:/Users/jjudge3/Desktop/Data/crossing_PPR/L23_vs_L4_NBQX/L4_NBQX_2/10-23-24/selected_zda/analysis01_02_03/custom_movie velocity v7.mp4
